# Grover Algorithm - Introduction

The [Grover algorithm](https://en.wikipedia.org/wiki/Grover%27s_algorithm?useskin=vector) is one of the most famous quantum algorithms. The algorithm got its name from Lov K. Grover, who wrote the [original paper](https://arxiv.org/abs/quant-ph/9605043) introducing this algorithm in 1996.

Grover's algorithm allows for searching unstructured datasets for solutions to a black box function.  The algorithm provides a quadratic speed-up over the best possible classical algorithm in terms of black box queries.

For further reading and watching, we recommend the [Qiskit tutorial](https://learn.qiskit.org/course/ch-algorithms/grovers-algorithm), this short [youtube video](https://www.youtube.com/watch?v=KeJqcnpPluc) introducing Grover's algorithm and the [Xanadu Codebook article](https://codebook.xanadu.ai/G.1).

This notebook teaches how to implement Grover's algorithm with [Pennylane](https://pennylane.ai/). If you get stuck at any point you can peek at the solutions provided in the end of this notebook, so that you can proceed with the subsequent tasks.

## The Problem

As mentioned in the introduction, Grover's algorithm is able to search for a solution in unstructured datasets. But what does this mean? Let's consider an example:

*Two people are playing a game. Person A thinks of an integer number between 1 and 100. Person B has to guess the number. But person B is only allowed to ask for a single number at a time and person A will only answer with "Yes, this is the number" or "No, this is not the number".*

The dataset in this example are all integer numbers 1-100. While in general these numbers of course have a structure, there is no relevant structure in the context of this game. By eliminating one number from the possible solutions (with a guess from B), no new information about the other numbers is gained. 

(**Side Note**: The situation would be different if person B would be allowed to ask e.g. "Is the number smaller than 40?". Then the order of the natural numbers would give a structure to the problem which could be exploited by the search algorithm.)

This is an example of a problem where finding the solution is hard but verifying a solution is easy (one just has to guess the correct number and A will confirm the solution). In this example person A plays the role of the back box or oracle function. In the classical case, B will need an average of $100/2=50$ guesses to hit the correct number. With Grover's algorithm (provided that the oracle can be formulated as a quantum operation) the correct number can be guessed *with high probability* in only about $\sqrt{100} = 10$ tries.


## Algorithm Structure

The Grover algorithm consists of three main parts:

1. **State preparation**
The initial state of the quantum register is prepared as a superposition of all possible solutions to the problem with equal weights. All solutions are equally likely to be measured. We will call this superposition state $|\psi\rangle$.

2. **Oracle function**
The oracle function is an operator which assigns a *negative phase* to the solution state(s) and leaves the other states untouched. As the phase is not physically measurable, all solutions are still equally likely. We will call the (unknown) solution state $|s\rangle$.

3. **Diffusion operator**
The diffusion operator amplifies the amplitude(s) of the solution state(s) by reflecting the state around $|\psi\rangle$. So it leaves the *uniform superposition part* of the state untouched and flips the sign of the amplitude of "everything else".

## Two Qubit Example Implementation

In [ ]:
#imports
import numpy as np
import pennylane as qml
from workshop.utils import sample_histogram

2 qubits have a $2^2 = 4$ dimensional hilbert space. So, we can play the guessing game with the numbers 1-4. We map each number to a qubit state:
- $1 \rightarrow |00\rangle$
- $2 \rightarrow |01\rangle$
- $3 \rightarrow |10\rangle$
- $4 \rightarrow |11\rangle$

We will assume that the solution is the number 4, i.e. the quantum state $|11\rangle$.

First, we initialize the circuit device with 2 qubits:

In [ ]:
device = qml.device("default.qubit", wires=2, shots=1000)

### State preparation

First, we want to prepare the 2-qubit quantum state in a way that all solutions are equally likely to be measures. 

**Task 1.1**: Fill in the code for the state preparation function in the cell below.

<details>
<summary>Click here for a hint</summary>
A superposition of all states with equal likelyhood can be created by applying Hadamard gates to all qubits.
</details>

In [ ]:
def state_preparation_gates():
    ################
    # Your Code Here
    ################


In [ ]:
@qml.qnode(device)
def state_preparation_circuit():
    state_preparation_gates()
    return qml.sample()

In [ ]:
qml.draw_mpl(state_preparation_circuit)()

In [ ]:
samples = state_preparation_circuit()
sample_histogram(samples)

The histogram shows that all states are sampled approximately equally often.

### Grover Oracle

As the oracle operator $\hat{U}_f$ we need an operator which does nothing to $|00\rangle, |01\rangle, |10\rangle$ and flips the sign of the amplitude of $|11\rangle$, i.e.

- $U_f |00\rangle = |00\rangle$
- $U_f |01\rangle = |01\rangle$
- $U_f |10\rangle = |10\rangle$
- $U_f |11\rangle = -|11\rangle$

An oracle function fulfilling these criteria is

$$ \hat{U}_f = I - 2 |11\rangle \langle 11| $$

**Task 1.2**: 
- (a) Verify that the oracle acts correctly on the 4 states.
- (b) Write the oracle in matrix notation

<details>
<summary>Click here for a hint for (a)</summary>
Remember that the basis states are orthogonal (i.e. $\langle X|Y\rangle = 0$ for $X \neq Y$) and that quantum states are normalized (i.e. $\langle X|X\rangle = 1$)
</details>

<details>
<summary>Click here for a hint for (b)</summary>
Remember that $|11\rangle = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix}$ and that $\langle 11| = \begin{bmatrix} 0,  0,  0,  1 \end{bmatrix}$
</details>

Compare the matrix representation of the oracle to the matrix representation of the [basic quantum gates](https://en.wikipedia.org/wiki/Quantum_logic_gate?useskin=vector) and find the correct representation for $\hat{U}_f$.

<details>
<summary>Click here for a hint</summary>
$\hat{U}_f$ can be represented with a single controlled Z (CZ gate).
</details>


**Task 1.3**: Implement the oracle in the cell below

In [ ]:
def oracle_gates():
    ################
    # Your Code Here
    ################
    

In [ ]:
@qml.qnode(device)
def oracle_circuit():
    oracle_gates()
    return qml.sample()

In [ ]:
qml.draw_mpl(oracle_circuit)()

**Task 1.4**: Verify that, after this step, all outcomes are still equally likely.

In [ ]:
################
# Your Code Here
################

### Diffusion operator

For the diffuser, we want to only flip the sign of the portion of the state which is perpendicular to the uniform superposition. The operator for that is

$$ \hat{D} = 2 |\psi \rangle \langle \psi| - I $$


This can be achived in terms of standard gates by the following steps:

- 1. Change the basis, so that the superposition state $|\psi\rangle$ and three states orthogonal to it are the new basis. We will do a rotation where we rotate $|\psi\rangle$ to the basis state $|00\rangle$. Remember that we started with the basis state $|00\rangle$ and obtained $|\psi\rangle$ by applying a Hadamard gate. So for rotating $|\psi\rangle$ to $|00\rangle$ we only have to invert the Hadamard gate.
- 2. flip the phase of all states orthogonal to $|00\rangle$
$$ \hat{U}_0 \frac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle) = \frac{1}{2}(|00\rangle - |01\rangle - |10\rangle - |11\rangle)$$
- 3. rotate back (undo step 1.)

**Optional Task**: 
- (a) Confirm with calculation on gate level that $\hat{U}_0 = \hat{CZ}\,(\hat{Z} \otimes \hat{Z})$ (with $\hat{CZ}$ being the controlled Z gate) fulfills the equation in step 2.

<details>
<summary>Click here for a hint</summary>
    Remember that $|00\rangle = |0\rangle \otimes |0\rangle$. The tensor product $\otimes$ splits the 2 qubit Hilbert space into the two single qubit Hilbert spaces.
    For an operator in the form $\hat{Z} \otimes \hat{Z}$, the first operator acts on the first qubit and the second operator acts on the second qubit.
    Furthermore, remember $\hat{Z} |0\rangle = |0\rangle$ and $\hat{Z} |1\rangle = -|1\rangle$.
</details>

**Solution**:
\begin{align*}
    \hat{U}_0 \frac{1}{2}(|00\rangle + |01\rangle + |10\rangle + |11\rangle) &= \frac{1}{2} \left( \hat{U}_0|00\rangle + \hat{U}_0|01\rangle + \hat{U}_0|10\rangle + \hat{U}_0|11\rangle\right)\\
    &= \frac{1}{2} \left( \hat{CZ}\,(\hat{Z} \otimes \hat{Z})|00\rangle + \hat{CZ}\,(\hat{Z} \otimes \hat{Z})|01\rangle + \hat{CZ}\,(\hat{Z} \otimes \hat{Z})|10\rangle + \hat{CZ}\,(\hat{Z} \otimes \hat{Z})|11\rangle\right)\\
    &= \frac{1}{2} \left( \hat{CZ}\,|00\rangle + \hat{CZ}\,(-|01\rangle) + \hat{CZ}\,(-|10\rangle) + \hat{CZ}\,|11\rangle\right)\\
    &= \frac{1}{2}(|00\rangle - |01\rangle - |10\rangle - |11\rangle)
\end{align*}

**Task 1.5 **: Implement the diffusion operator in the function below

<details>
<summary>Click here for a hint</summary>
    The rotation in step 1. can be achieved ba simply applying Hadamard gates to all qubits. Remember that the effect of an Hadamard gate can be inverted by applying another Hadamard gate. Applying the Hadamard gate in step 1. changes the basis to the uniform superposition state and the states orthogonal to it. The Hadamard gate in step 3. changes the basis back to the original basis.
    The full Diffusion operator consists of $\hat{H}\hat{U}_0\hat{H}$, i.e. 
    \begin{align*}
        (\hat{H} \otimes \hat{H}) \, \hat{CZ}\,(\hat{Z} \otimes \hat{Z})\, (\hat{H} \otimes \hat{H})
    \end{align*}
    Remember to apply the gates in the correct order. the rightmost operators in the formular are applied first.
    The operators on the lefthand side of the tensor product $\otimes$ are applied to the first qubit and the operators on the righthand side to the second qubit. The control qubit of the CZ gat is the first qubit and it acts on the second qubit.
</details>

In [ ]:
def diffusion_gates():
    ################
    # Your Code Here
    ################


In [ ]:
@qml.qnode(device)
def diffusion_circuit():
    diffusion_gates()
    return qml.sample()

In [ ]:
qml.draw_mpl(diffusion_circuit)()

### Putting everything together

Finally, we can put all parts together in a single circuit. For obtaining the result we actually only need a single shot on the quantum device. So we define a new single shot quantum device in the next cell.

In [ ]:
single_shot_device = qml.device("default.qubit", wires=2, shots=1)

In [ ]:
@qml.qnode(single_shot_device)
def grover_circuit():
    
    state_preparation_gates()
    oracle_gates()
    diffusion_gates()
    
    return qml.sample()

In [ ]:
qml.draw_mpl(grover_circuit)()

In [ ]:
samples = grover_circuit()
sample_histogram(np.atleast_2d(samples))

If we run the circuit now, we see that the correct solution is almost with probability 100% the result of the measurement. So we can get the result with only a single guess via the oracle function!

With larger problem sizes more guesses are needed to reach a high probability. This is being done by repeating grover iterations consisting of the oracle function and the diffusion operation. The speedup stems from the fact that the needed Grover iterations scale only with the square root of $N$. The exact formula for the number of needed guesses in terms of the problem size is discussed in the next excercise notebook.

## Higher Dimensional Example

For a better understanding of the implementation details of the algorithm, we generalize the implementation from the 2 qubit example to the n-qubit case in the following.

**Note**: Only the oracle function (which is solution dependent) generally has to be edited when changing the dimensionality or the solution state.

The Hilbert space (state space) of n qubits is $2^n$ dimensional. Hence, for $n=3$ qubits, the guessing game can be played with up to 8 numbers. For $n=4$ with 16 numbers, for $n=5$ with 32 numbers, etc.

In [ ]:
# define number of qubits and a device for running the circuit
n_qubits = 4
single_shot_device = qml.device("default.qubit", wires=n_qubits, shots=1)

### State Preparation

**Task 2.1**: Implement the n-qubit state preparation so that all states have equal probability to be measured.

<details>
<summary>Click here for a hint</summary>
A superposition of all states with equal likelihood can be created by applying Hadamard gates to all qubits.
</details>

In [ ]:
def state_preparation_gates():
    ################
    # Your Code Here
    ################
    

### Oracle

The implementation of the oracle needs a bit more care. In the 2 qubit case, we derived the oracle circuit from the matrix representation of the operator which is not so difficult to guess ($\hat{U}_f = I - |solution\rangle \langle solution|$). However, the derivation with the matrix representation is a bit cumbersome. But with the knowlege of the functionality of the gates, and a little thinking, we can construct the oracle function for higher dimensions as well.

Remember that the oracle flips the sign of the solution state while leaving the other states untouched. Let us say that for 4 qubits we are searching for the solution $|1001\rangle$ (number 9 in the guessing game).

After the state preparation we have the equal probability state

\begin{align*}
    |\psi\rangle = \frac{1}{\sqrt{2^{4}}} \left( |0000\rangle + |0001\rangle + ... + |1111\rangle \right)
\end{align*}

and now we are looking for an operator $\hat{U}_f$, for which

\begin{align*}
    \hat{U}_f|1001\rangle &= -|1001\rangle \\
    \hat{U}_f|x\rangle &= |x\rangle \quad \text{for} \quad |x\rangle \neq |1001\rangle
\end{align*}

This can be achieved by applying a MultiControlled Z gate to the last qubit. Remember that the Z gate only adds a minus sign to the portion of the state where the qubit it acts on is in state $|1\rangle$. So if the Z gate is applied to the last qubit, there is no need to check for the state of the last qubit as the Z gate only affects the "1" portion anyways ($|1000\rangle$ will not be changed). So what is needed here is a MultiControlled Z gate which cheks the states of the first three qubits and acts on the fourth.

**Task 2.2**: Implement the Oracle for the 4 qubit circuit and solution state $|1001\rangle$.

**Free Hint**: Pennylane only features a [MultiControlledX](https://docs.pennylane.ai/en/stable/code/api/pennylane.MultiControlledX.html) gate (no MultiControlledZ gate). But you can use the fact that $\hat{H}\hat{X}\hat{H} = \hat{Z}$ (You can verify this easily in the matrix representation).

In [ ]:
def oracle_gates():
    ################
    # Your Code Here
    ################
    

**Note**: As the solution state changes with the dimensionality of the problem, the oracle needs slight modification when the number of qubits changes.

**Note**: If you want to act with the Z Gate on a qubit state with a 0 (e.g. if the solution is the $|00..0\rangle$ state), you have to flip the qubit the gate acts on first with an X gate, then apply the Z gate and afterwards undo the initial flip with another X gate.

### Diffusor

The Diffusor can be written in a general form for n qubits. Remember that the diffusor is supposed to change the sign of all states perpendicular to the start state (or after rotating the start state back to the zero state: change the sign of all but the $|00...0\rangle$ state). 

**Note**: *Changing the sign of all states but the zero state is the same as changing only the sign of the zero state*. A global minus sign (a global phase) does not influence the measurement (in fact it cannot be measured at all!).

The plan for the general diffusor is:

1. rotate the state back to the zero state
2. change the sign of the $|00...0\rangle$ state
3. rotate back

**Task 2.3**: Implement the general diffusor.

<details>
<summary>Click here for a hint</summary>
The rotation again can be achieved by applying Hadamard gates to all qubits.
</details>

<details>
<summary>Click here for a hint</summary>
As we want to change the sign of the $|00...0\rangle$ state and the easiest way to change the sign is with a $\hat{Z}$ gate, it is convenient to first invert the state, i.e. convert all 0 states to 1 and all 1 states to 0. This can be achieved by applying $\hat{X}$ gates to all qubits. Then one can flip the sign of the $|11...1\rangle$ state with a $\hat{Z}$ gate. Remember to undo this conversion after the sign flipping by again applying $\hat{X}$ gates.
</details>

<details>
<summary>Click here for a hint</summary>
For flipping the sign of the $|1...1\rangle$ state, you can again use a MultiControlledZ gate to make sure that the Z gate only effects the $|1...1\rangle$ basis state.
</details>

In [ ]:
def diffusion_gates():
    ################
    # Your Code Here
    ################
    

### Everything together

For running the algorithm one also needs to know the number of grover iterations which are needed for maximizing the probability of the solution. The formula is provided in the next cell but will be discussed in more detail in the next notebook.

In [ ]:
grover_iterations = int(np.ceil(np.sqrt(2**n_qubits)*np.pi/4.0 - 0.5))
print(f"Using {grover_iterations} Grover iterations")

In [ ]:
@qml.qnode(single_shot_device)
def grover_circuit():
    
    state_preparation_gates()
    for _ in range(grover_iterations):
        oracle_gates()
        diffusion_gates()
    
    return qml.sample()

In [ ]:
qml.draw_mpl(grover_circuit)()

In [ ]:
samples = grover_circuit()
print(samples)

A single shot of the circuit should give you the solution with almost 100% probability. (Verify by checking a few samples or by increasing the shots and plotting a histogram.)

## Conclusion

The key to the Grover algorithm is the oracle function and some clever splitting, rotating and reflecting of the quantum state. Looking at the oracle function in this example here, you may have noticed that it was necessary to know the solution to implement the oracle function. And in this example game this is true, as the solution can only be verified by knowing the solution. However, there are also problems where the solution can be verified without knowing the solution explicitly. You will solve such a problem in the next notebook.

# Solutions

<details>
<summary>Solution 1.1</summary>
<code>
def state_preparation_gates():
    ################
    # Your Code Here
    ################
    # Apply the Hadamard gate to both qubits
    for i in range(2):
        qml.Hadamard(wires=i)
</code>
</details>

<details>
<summary>Solution 1.2 (a)</summary>
\begin{align*}
        \hat{U}_f |00\rangle &= \left( I - 2 |11\rangle \langle 11| \right) |00\rangle\\
                             &=  |00\rangle - 2 |11\rangle \langle 11|00\rangle\\
                             &= |00\rangle
\end{align*}
    
Note that $\langle 11|00\rangle = 0$ because the states are orthogonal. The same holds for $|01\rangle$ and $|10\rangle$.

For $|11\rangle$:
    
\begin{align*}
    \hat{U}_f |11\rangle &= \left( I - 2 |11\rangle \langle 11| \right) |11\rangle\\
                         &= |11\rangle - 2 |11\rangle \langle 11|11\rangle\\
                         &= |11\rangle - 2 |11\rangle \\
                         &= -|11\rangle
\end{align*}

Note that $\langle 11|11\rangle = 1$ as every quantum state is normalized.
</details>

<details>
<summary>Solution 1.2 (b)</summary>
In the 4 dimensional vector representation of the Hilbert space, the basis vector $|11\rangle$ can be written as

\begin{align*}
|11\rangle = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix}
\end{align*}

and $\langle 11|$ is the complex conjugate of this vector:

\begin{align*}
\langle 11| = \begin{bmatrix} 0,  0,  0,  1 \end{bmatrix}
\end{align*}

Putting these together yields:

\begin{align*}
    \hat{U}_f &= I - 2 |11\rangle \langle 11| \\
               &= \begin{bmatrix} 1, 0, 0, 0 \\
                                 0, 1, 0, 0 \\
                                 0, 0, 1, 0 \\
                                 0, 0, 0, 1 \end{bmatrix}
                - 2 \begin{bmatrix} 0 \\ 0 \\ 0 \\ 1 \end{bmatrix} \begin{bmatrix} 0,  0,  0,  1 \end{bmatrix}\\
            &= \begin{bmatrix} 1, 0, 0, 0 \\
                               0, 1, 0, 0 \\
                               0, 0, 1, 0 \\
                               0, 0, 0, 1 \end{bmatrix}
                - \begin{bmatrix} 0, 0, 0, 0 \\
                                  0, 0, 0, 0 \\
                                  0, 0, 0, 0 \\
                                  0, 0, 0, 2 \end{bmatrix}\\
            &=  \begin{bmatrix} 1, 0, 0, 0 \\
                                0, 1, 0, 0 \\
                                0, 0, 1, 0 \\
                                0, 0, 0, -1 \end{bmatrix}
\end{align*}
</details>

<details>
<summary>Solution 1.3</summary>
<code>
def oracle_gates():
    ################
    # Your Code Here
    ################
    qml.CZ([0, 1])
</code>
</details>

<details>
<summary>Solution 1.4</summary>
<code>
################
# Your Code Here
################

@qml.qnode(device)
def preparation_and_oracle_circuit():
    state_preparation_gates()
    oracle_gates()
    return qml.sample()

qml.draw_mpl(preparation_and_oracle_circuit)()
sample_histogram(preparation_and_oracle_circuit())
</code>
</details>

<details>
<summary>Solution 1.5</summary>
<code>
def diffusion_gates():
    ################
    # Your Code Here
    ################
    for i in range(2):
        qml.Hadamard(i)
        qml.PauliZ(i)    
    qml.CZ([0, 1])
    for i in range(2):
        qml.Hadamard(i)
</code>
</details>

<details>
<summary>Solution 2.1</summary>
<code>
def state_preparation_gates():
    ################
    # Your Code Here
    ################
    # Apply the Hadamard gate to all qubits
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
</code>
</details>

<details>
<summary>Solution 2.2</summary>
<code>
def oracle_gates():
    ################
    # Your Code Here
    ################
    qml.Hadamard(n_qubits-1)
    qml.MultiControlledX(wires=[0, 1, 2, 3], control_values="100")
    qml.Hadamard(n_qubits-1)
</code>
</details>

<details>
<summary>Solution 2.3</summary>
<code>
def diffusion_gates():
    ################
    # Your Code Here
    ################
    for i in range(n_qubits):
        qml.Hadamard(i)
    for i in range(n_qubits):
        qml.PauliX(i)  
    qml.Hadamard(n_qubits-1)
    qml.MultiControlledX(wires=[i for i in range(n_qubits)])
    qml.Hadamard(n_qubits-1)
    for i in range(n_qubits):
        qml.PauliX(i)
    for i in range(n_qubits):
        qml.Hadamard(i)
</code>
</details>